作るべき函数

相対誤差 abs(answer/solution - 1) が 0.005 以下なら正解とする.

解答用紙生成函数: 辞書を返す

採点結果生成函数: 辞書と点数を返す

In [35]:
using TOML, Unicode, PrettyPrinting

In [188]:
input = """
[1]
[1.1]
x = [123.4, 5]
y = ["yes", 5]
z = [[34.5, 199.5], 10]
[1.2]
w = [999, 10]

[2]
a = [678, 5]
b = [1003, 5]

[3]
c = [123, 20]
"""
println(input)

data = TOML.parse(input)
pprint(data)

[1]
[1.1]
x = [123.4, 5]
y = ["yes", 5]
z = [[34.5, 199.5], 10]
[1.2]
w = [999, 10]

[2]
a = [678, 5]
b = [1003, 5]

[3]
c = [123, 20]

Dict("1" => Dict("1" => Dict("x" => [123.4, 5],
                             "z" => [[34.5, 199.5], 10],
                             "y" => ["yes", 5]),
                 "2" => Dict("w" => [999, 10])),
     "2" => Dict("b" => [1003, 5], "a" => [678, 5]),
     "3" => Dict("c" => [123, 20]))

In [142]:
input1 = """
name = "山田太郎"
id = "A9BC99999"

[1]
[1.1]
x = 123.40001
y = "yes"
z = [34.500001, 199.500001]
[1.2]
w = 999

[2]
a = 678
b = 1003
"""
println(input1)

data1 = TOML.parse(input1)
pprint(data1)

name = "山田太郎"
id = "A9BC99999"

[1]
[1.1]
x = 123.40001
y = "yes"
z = [34.500001, 199.500001]
[1.2]
w = 999

[2]
a = 678
b = 1003

Dict("name" => "山田太郎",
     "1" => Dict("1" => Dict("x" => 123.40001,
                             "z" => [34.500001, 199.500001],
                             "y" => "yes"),
                 "2" => Dict("w" => 999)),
     "2" => Dict("b" => 1003, "a" => 678),
     "id" => "A9BC99999")

In [143]:
input2 = """
name = "山田太郎"
id = "A9BC99999"

[1]
[1.1]
y = "Yes"
z = [34.500001, 199.500001]
[1.2]
w = 990

[2]
a = 678
b = 1003
"""
println(input2)

data2 = TOML.parse(input2)
pprint(data2)

name = "山田太郎"
id = "A9BC99999"

[1]
[1.1]
y = "Yes"
z = [34.500001, 199.500001]
[1.2]
w = 990

[2]
a = 678
b = 1003

Dict("name" => "山田太郎",
     "1" => Dict("1" => Dict("z" => [34.500001, 199.500001], "y" => "Yes"),
                 "2" => Dict("w" => 990)),
     "2" => Dict("b" => 1003, "a" => 678),
     "id" => "A9BC99999")

In [144]:
input3 = """
name = "山田太郎"
id = "A9BC99999"

[1]
[1.1]
z = [34.500001, 199.500001]
[1.2]
w = 990

[2]
a = "yes"
b = 1003
"""
println(input3)

data3 = TOML.parse(input3)
pprint(data3)

name = "山田太郎"
id = "A9BC99999"

[1]
[1.1]
z = [34.500001, 199.500001]
[1.2]
w = 990

[2]
a = "yes"
b = 1003

Dict("name" => "山田太郎",
     "1" => Dict("1" => Dict("z" => [34.500001, 199.500001]),
                 "2" => Dict("w" => 990)),
     "2" => Dict("b" => 1003, "a" => "yes"),
     "id" => "A9BC99999")

In [152]:
relerr(x, y) = Inf
relerr(x::Real, y::Real) = abs(x/y - 1)
relerr(x::Vector, y::Vector) = maximum(relerr(x[i], y[i]) for i in eachindex(y))
relerr(x::AbstractString, y::AbstractString) =
    Unicode.normalize(x; casefold=true) != Unicode.normalize(y; casefold=true)
nearlyequal(x, y; reltol = 0.005) = relerr(x, y) ≤ reltol

kukaito(x) = ""
kukaito(x::Dict) = Dict()
kukaito(x::AbstractFloat) = NaN

function saiten(ans, sol::Vector, gokeiten = 0)
    score = nearlyequal(ans, sol[1]) * sol[2]
    Union{typeof(ans), typeof(score)}[ans, score], gokeiten + score
end

function saiten(ans::Dict, sol::Dict, gokeiten = 0)
    kekka = Dict()
    for k in keys(sol)
        v, gokeiten = saiten(get(ans, k, kukaito(sol[k])), sol[k], gokeiten)
        kekka[k] = v
    end
    kekka, gokeiten
end

saiten (generic function with 4 methods)

In [153]:
saiten(123.1, [123, 10], 0)

(Union{Float64, Int64}[123.1, 10], 10)

In [154]:
kekka1, gokeiten1 = saiten(data1, data)
pprint(kekka1)
gokeiten1

Dict("1" => Dict("1" => Dict("x" => [123.40001, 5],
                             "z" => [[34.500001, 199.500001], 10],
                             "y" => ["yes", 5]),
                 "2" => Dict("w" => [999, 10])),
     "2" => Dict("b" => [1003, 5], "a" => [678, 5]),
     "3" => Dict("c" => ["", 0]))

40

In [155]:
TOML.print(kekka1; sorted=true)


[1.1]
x = [123.40001, 5]
y = ["yes", 5]
z = [[34.500001, 199.500001], 10]

[1.2]
w = [999, 10]
[2]
a = [678, 5]
b = [1003, 5]

[3]
c = ["", 0]


In [156]:
kekka2, gokeiten2 = saiten(data2, data)
pprint(kekka2)
gokeiten2

Dict("1" => Dict("1" => Dict("x" => ["", 0],
                             "z" => [[34.500001, 199.500001], 10],
                             "y" => ["Yes", 5]),
                 "2" => Dict("w" => [990, 0])),
     "2" => Dict("b" => [1003, 5], "a" => [678, 5]),
     "3" => Dict("c" => ["", 0]))

25

In [157]:
TOML.print(kekka2; sorted=true)


[1.1]
x = ["", 0]
y = ["Yes", 5]
z = [[34.500001, 199.500001], 10]

[1.2]
w = [990, 0]
[2]
a = [678, 5]
b = [1003, 5]

[3]
c = ["", 0]


In [158]:
kekka3, gokeiten3 = saiten(data3, data)
pprint(kekka3)
gokeiten3

Dict("1" => Dict("1" => Dict("x" => ["", 0],
                             "z" => [[34.500001, 199.500001], 10],
                             "y" => ["", 0]),
                 "2" => Dict("w" => [990, 0])),
     "2" => Dict("b" => [1003, 5], "a" => ["yes", 0]),
     "3" => Dict("c" => ["", 0]))

15

In [159]:
TOML.print(kekka3; sorted=true)


[1.1]
x = ["", 0]
y = ["", 0]
z = [[34.500001, 199.500001], 10]

[1.2]
w = [990, 0]
[2]
a = ["yes", 0]
b = [1003, 5]

[3]
c = ["", 0]


In [160]:
pprint(data3)

Dict("name" => "山田太郎",
     "1" => Dict("1" => Dict("z" => [34.500001, 199.500001]),
                 "2" => Dict("w" => 990)),
     "2" => Dict("b" => 1003, "a" => "yes"),
     "id" => "A9BC99999")

In [182]:
gijikaito(x::AbstractFloat) = 46.49373
gijikaito(x::Integer) = 4649373
gijikaito(x::AbstractString) = "yes or no"
gijikaito(x::Vector) = [gijikaito(x) for x in x]

make_kaitoyoshi(sol::Vector; gijikaitofunc=gijikaito) = gijikaitofunc(sol[1])

function make_kaitoyoshi(sol::Dict; gijikaitofunc=gijikaito)
    kaitoyoshi = Dict()
    for k in keys(sol)
        v = make_kaitoyoshi(sol[k]; gijikaitofunc)
        kaitoyoshi[k] = v
    end
    kaitoyoshi
end

make_mohankaito(sol::Dict) = make_kaitoyoshi(sol; gijikaitofunc=identity)

make_mohankaito (generic function with 1 method)

In [183]:
mohankaito = make_mohankaito(data)
TOML.print(mohankaito)


[1.1]
x = 123.4
z = [34.5, 199.5]
y = "yes"

[1.2]
w = 999
[2]
b = 1003
a = 678

[3]
c = 123


In [187]:
TOML.print(data)


[1.1]
x = [123.4, 5]
z = [[34.5, 199.5], 10]
y = ["yes", 5]

[1.2]
w = [999, 10]
[2]
b = [1003, 5]
a = [678, 5]

[3]
c = [123, 20]


In [174]:
kaitoyoshi = make_kaitoyoshi(data)
pprint(kaitoyoshi)

Dict("1" => Dict("1" => Dict("x" => 46.49373,
                             "z" => [46.49373, 46.49373],
                             "y" => "yes or no"),
                 "2" => Dict("w" => 4649373)),
     "2" => Dict("b" => 4649373, "a" => 4649373),
     "3" => Dict("c" => 4649373))

In [175]:
TOML.print(kaitoyoshi; sorted=true)


[1.1]
x = 46.49373
y = "yes or no"
z = [46.49373, 46.49373]

[1.2]
w = 4649373
[2]
a = 4649373
b = 4649373

[3]
c = 4649373


In [176]:
TOML.print(data; sorted=true)


[1.1]
x = [123.4, 5]
y = ["yes", 5]
z = [[34.5, 199.5], 10]

[1.2]
w = [999, 10]
[2]
a = [678, 5]
b = [1003, 5]

[3]
c = [123, 20]


In [172]:
pprintln.(saiten(kaitoyoshi, data));

Dict("1" => Dict("1" => Dict("x" => [46.49373, 0],
                             "z" => [[46.49373, 46.49373], 0],
                             "y" => ["yes or no", 0]),
                 "2" => Dict("w" => [4649373, 0])),
     "2" => Dict("b" => [4649373, 0], "a" => [4649373, 0]),
     "3" => Dict("c" => [4649373, 0]))
0
